In [9]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.constants._master import Master
from src.constants._results_cols import ResultsCols as Cols
from src.preparing.DataLoader import DataLoader
from src.preparing.prepare_chrome_driver import prepare_chrome_driver

%load_ext autoreload

In [106]:
from src.preparing.table_creator import TableCreator
from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup


def create_race_results_table_():
    race_html_list = ["202001010101.bin"]#, "202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            html = f.read()
            # メインとなるレース結果テーブルデータを取得
            df = pd.read_html(html)
            print("df:", df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            print("soup: ", soup)


alias = "race_results_table"
data_request = TableCreator()
data_request.set_args(alias)
data_request.pre_process_display()
create_race_results_table_()

race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
race_results_table
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/race_results/
to_location:  ./data/html/race_results/
self.from_local_location:  ./data/html/race/
self.from_local_file_name:  *.bin
batch_size: 100
reloaded_*.bin type:  *.bin
creating race_results_table


100%|██████████| 1/1 [00:00<00:00,  6.62it/s]

df: [   着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差    単勝  人 気      馬体重  \
0    1    6    6     ウインルーア  牝2  54  横山武史  1:49.7    NaN  16.0    3  438(+4)   
1    2    2    2     アークライト  牡2  54  ルメール  1:50.0  1.3/4   1.9    2   510(0)   
2    3    3    3  ギャラントウォリア  牡2  54  池添謙一  1:50.1    1/2   1.8    1  482(-6)   
3    4    1    1     ジュンブーケ  牝2  52  亀田温心  1:50.5  2.1/2  22.2    4   442(0)   
4    5    4    4    キタノマンゲツ  牡2  54  藤岡康太  1:51.0      3  55.7    5  426(-8)   
5    6    5    5     アイフレンズ  牝2  54   黛弘人  1:52.7      大  73.9    6  426(-2)   

        調教師  
0  [東] 青木孝文  
1  [東] 藤沢和雄  
2   [西] 平田修  
3   [西] 森秀行  
4  [西] 中尾秀正  
5  [東] 中野栄治  ,     0      1        2    3
0  単勝      6     1600    3
1  複勝    6 2  490 130  4 2
2  馬連  2 - 6     1410    4,      0                  1            2      3
0  ワイド  2 - 6 3 - 6 2 - 3  150 170 110  3 4 1
1   馬単              6 → 2         3920      9
2  三連複          2 - 3 - 6          240      2
3  三連単          6 → 2 → 3         5180    

In [31]:
# from src.preparing.table_creator import TableCreator
# from src.preparing.url_loader import KaisaiDateLoader
from tqdm import tqdm
import os
import pandas as pd
from bs4 import BeautifulSoup
import re
from src.constants._master import Master


def create_race_info_table_():
    race_html_list = ["202001010101.bin", "202001010102.bin", "202001010103.bin"]

    print("creating race_results_table")

    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            html = f.read()
            #df = pd.read_html(html)
            # print("df:",df)
            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            # print("soup: ",soup)
            # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            # print("texts: ",texts)
            info = re.findall(r"\w+", texts)
            # print("info: ",info)

            df = pd.DataFrame()
            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text]
                if "障" in text:
                    df["race_type"] = ["障害"]
                    hurdle_race_flg = True
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])]
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text]
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text]
                if "年" in text:
                    df["date"] = [text]
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]]
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]]
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]]
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]]
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]]
                if ("1勝クラス" in text) or ("500万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]]
                if ("2勝クラス" in text) or ("1000万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]]
                if ("3勝クラス" in text) or ("1600万下" in text):
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]]
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]]
            # グレードレース情報の取得
            grade_text = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
            if "G3" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif "G2" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif "G1" in grade_text:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]]
                df["race_class"] = [Master.RACE_CLASS_LIST[9]]

            # インデックスをrace_idにする
            race_id = re.findall(r"(\d+).bin", race_html)[0]
            df.index = [race_id] * len(df)
            race_infos = {}
            race_infos[race_id] = df
    race_infos_df = pd.concat([race_infos[key] for key in race_infos])
    print(race_infos_df)


# alias = "race_results_table"
# data_request = TableCreator()
# data_request.set_args(alias)
# data_request.pre_process_display()
create_race_info_table_()

creating race_results_table


100%|██████████| 3/3 [00:00<00:00, 15.91it/s]

              course_len around weather race_type ground_state        date  \
202001010103        1500      右       曇         芝            良  2020年7月25日   

             race_class  
202001010103        未勝利  


In [65]:
import time
from urllib.request import urlopen

def scrape_horse_id_list_():
    """
    当日出走するhorse_id一覧を取得
    """

    print("scraping horse_id_list")
    race_id_list =["202001010101","202001010102", "202001010103"]
    horse_id_list=[]
    for race_id in tqdm(race_id_list):
        # wait = WebDriverWait(driver, waiting_time)
        try:
            url = "https://db.netkeiba.com/race/"+ race_id

            # wait.until(EC.presence_of_element_located((By.CLASS_NAME, "HorseInfo")))
            html = urlopen(url)
            #soup = BeautifulSoup(html, "lxml", from_encoding="utf-8")
            soup = BeautifulSoup(html, "lxml")
            #print("soup",soup)
            horse_td_list = soup.find_all("td", attrs={"class": "txt_l"})
            horse_ids =[a['href'] for td in horse_td_list if (a := td.find('a')) and '/horse/' in a['href']]
            horse_id = [re.search(r'/horse/(\d+)/', href).group(1)  for href in horse_ids]
            horse_id_list.append(horse_id)
            # 内側のリストを外す
            flat_list = [item for sublist in horse_id_list for item in sublist]
            target_data = sorted(set(flat_list))
        except Exception as e:
            print(e)
            break
    print("horse_td_list:",target_data)



scrape_horse_id_list_()

scraping horse_id_list


100%|██████████| 3/3 [00:01<00:00,  2.49it/s]

horse_td_list: ['2017100432', '2017100813', '2017101364', '2017101757', '2017102190', '2017102915', '2017102918', '2017103355', '2017103622', '2017103709', '2017103841', '2017103915', '2017103953', '2017103996', '2017104065', '2017104160', '2017104334', '2017104792', '2017104859', '2017104882', '2017105136', '2017105153', '2017105242', '2017105261', '2017105324', '2017106065', '2017106103', '2017106389', '2018100828', '2018101626', '2018102410', '2018104800', '2018105193', '2018106434']


In [93]:
import datetime
import os
import re
import time
from urllib.parse import urlencode
from urllib.request import urlopen

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

from src.preparing.DataLoader import DataLoader
from src.preparing.prepare_chrome_driver import prepare_chrome_driver

def scrape_peds_list_():
    """
    horse/pedページのhtmlを受け取って、血統のDataFrameに変換する関数。
    """
    # skip対象ではないゴミファイルの掃除

    ped_html_list = ["2018102657.bin"]#,"2017100282.bin"]
    data_index = 1
    peds = {}
    print("preparing peds table")
    for ped_html in tqdm(ped_html_list):
        ped_html_path = os.path.join("./data/html/ped/", ped_html)
        # print("ped_html_path", ped_html_path)
        try:
            with open(ped_html_path, "rb") as f:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                # horse_idを取得
                #print("ped_html",ped_html)
                horse_id = re.findall(r"(\d+)", ped_html)[0]
                print("horse_id",horse_id)
                # htmlをsoupオブジェクトに変換
                soup = BeautifulSoup(html, "lxml")
                #print("soup",soup)
                peds_id_list = []
                # 血統データからhorse_idを取得する
                horse_a_list = soup.find("table", attrs={"summary": "5代血統表"}).find_all(
                    "a", attrs={"href": re.compile(r"^/horse/\w{10}")}
                )

                for a in horse_a_list:
                    # 血統データのhorse_idを抜き出す
                    work_peds_id = re.findall(r"horse\W(\w{10})", a["href"])[0]
                    print("work_peds_id",work_peds_id)
                    peds_id_list.append(work_peds_id)
                print("peds_id_list",peds_id_list)
                peds[horse_id] = peds_id_list
                print("peds",peds[horse_id] )
                # pd.DataFrame型にして一つのデータにまとめて、列と行の入れ替えして、列名をpeds_0, ..., peds_61にする
                peds_df = pd.DataFrame.from_dict(peds, orient="index").add_prefix("peds_")
                print("peds_df",peds_df)
            #if data_index % self.batch_size == 0:
            #    self.target_data = peds_df
            #    self.save_temp_file("peds_list")
            #    self.obtained_last_key = ped_html
        except Exception as e:
            print("Error at {}: {}".format(ped_html, e))
        #data_index = +1

scrape_peds_list_()

preparing peds table


100%|██████████| 1/1 [00:00<00:00, 17.78it/s]

horse_id 2018102657
work_peds_id 2012104511
work_peds_id 2001103460
work_peds_id 000a001d7e
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a007e0c
work_peds_id 000a009961
work_peds_id 000a001676
work_peds_id 000a0084c9
work_peds_id 000a00680e
work_peds_id 000a0001d6
work_peds_id 000a000378
work_peds_id 000a009241
work_peds_id 000a00b1e0
work_peds_id 000a000e1d
work_peds_id 000a009608
work_peds_id 2000107542
work_peds_id 000a00033a
work_peds_id 000a0012bf
work_peds_id 000a000f2b
work_peds_id 000a007459
work_peds_id 000a008c1e
work_peds_id 000a0019b6
work_peds_id 000a008c1d
work_peds_id 1993109154
work_peds_id 1983109006
work_peds_id 000a000827
work_peds_id 000a00859c
work_peds_id 1980101983
work_peds_id 000a000258
work_peds_id 1955105713
work_peds_id 2003101283
work_peds_id 000a000134
work_peds_id 000a001607
work_peds_id 000a000e46
work_peds_id 000a000f89
work_peds_id 000a007984
work_peds_id 000a007e0c
work_peds_id 000a000f87
work_peds_id 000a007d1c
work_peds_id 000a009

In [118]:


def create_horse_results_table_():
    """
    horse_htmlを受け取って、結果テーブルに変換する関数。data/html/horse_resultsに保存する

    """

    horse_html_list = ['2017103612.bin','2015105131.bin']
    data_index = 1
    print("creating horse_results_table")
    horse_results = {}
    horse_id_df = pd.DataFrame(columns=["horse_id"])
    for horse_html in tqdm(horse_html_list):
        #horse_html_path = os.path.join('./data/tmp/horse', horse_html)
        #with open(horse_html_path, "rb") as f:
        try:
            # 保存してあるbinファイルを読み込む
            #html = f.read
            # パスから正規表現でhorse_id_listを取得
            #print("horse_html",horse_html)
            #print("horse_html_path",horse_html_path)

            horse_id_list = re.findall(r"(\d+).bin", horse_html)
            #print("horse_id_list",horse_id_list)
            # DataFrameに追加
            horse_id_df = pd.concat([horse_id_df, pd.DataFrame({"horse_id": horse_id_list})], ignore_index=True)
            print("horse_id_df ", horse_id_df)

            ### 取得日マスタの更新 ###
            print("updating master")
            now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 現在日時を取得

            pd.DataFrame(columns=["horse_id", "updated_at"])
            # マスタを読み込み
            master = pd.read_csv("horse_results.pickle", dtype=object)
            # horse_id列に新しい馬を追加
            #new_master = master.merge(horse_id_df, on="horse_id", how="outer")
            # マスタ更新
            #new_master.loc[new_master["horse_id"].isin(horse_id_list), "updated_at"] = now
            # 列が入れ替わってしまう場合があるので、修正しつつ保存
            #target_data = new_master[["horse_id", "updated_at"]]
            #obtained_last_key = horse_html
        except Exception as e:
            print("Error at {}: {}".format(horse_html_path, e))

        data_index += 1


create_horse_results_table_()


creating horse_results_table


100%|██████████| 2/2 [00:00<00:00, 453.49it/s]

horse_id_df       horse_id
0  2017103612
updating master
horse_id_df       horse_id
0  2017103612
1  2015105131
updating master


In [131]:
def create_horse_results_table_():
    #
    # horseページのhtmlを受け取って、馬の過去成績のDataFrameに変換する関数。
    horse_html_list = ['2017103612.bin']#,'2015105131.bin']
    data_index = 1
    horse_id_file_list = './data/tmp/horse'

    print("scraping horse_results_table")
    horse_results = {}
    df=pd.DataFrame()
    for horse_html in tqdm(horse_html_list):
        horse_html_path = os.path.join('./data/tmp/horse', horse_html)
        with open(horse_html_path, "rb") as f:
            try:
                # 保存してあるbinファイルを読み込む
                html = f.read()
                #print("html",html)
                df = pd.read_html(html)[3]
                print("df",df)
                # 受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
                if df.columns[0] == "受賞歴":
                    df = pd.read_html(html)[4]
                    print("df4",df)
                # 新馬の競走馬レビューが付いた場合、
                # 列名に0が付与されるため、次のhtmlへ飛ばす
                if df.columns[0] == 0:
                    print("horse_results empty case1 {}".format(horse_html))
                    continue

                horse_id = re.findall(r"(\d+).bin", horse_html)[0]
                print("horse_id",horse_id)
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df

            # 競走データが無い場合（新馬）を飛ばす
            except IndexError:
                print("horse_results empty case2 {}".format(horse_html))
                continue

            except Exception as e:
                print("Error at {}: {}".format(horse_html, e))

        # pd.DataFrame型にして一つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        # 列名に半角スペースがあれば除去する
    target_data = horse_results_df.rename(columns=lambda x: x.replace(" ", ""))
    print("target",target_data)



create_horse_results_table_()

scraping horse_results_table


100%|██████████| 1/1 [00:00<00:00, 42.19it/s]

df            日付    開催 天 気  R   レース名  映 像  頭 数  枠 番  馬 番  オ ッ ズ  ...   着差  \
0  2020/08/30  2小倉6   晴  6  3歳未勝利  NaN   18    7   13   10.1  ...  3.7   
1  2020/08/15  2小倉1   晴  7  3歳未勝利  NaN   18    5    9   20.5  ...  0.2   
2  2020/07/12  4阪神4   晴  3  3歳未勝利  NaN   16    4    7   65.5  ...  2.5   
3  2020/04/18  2阪神7   曇  4  3歳未勝利  NaN   18    2    3   25.0  ...  3.1   
4  2020/03/20  1阪神7   晴  6  3歳未勝利  NaN   16    8   16   29.7  ...  1.5   
5  2020/02/29  1中京1   曇  4  3歳未勝利  NaN   17    3    6   17.1  ...  0.6   
6  2020/02/09  2京都4   曇  6   3歳新馬  NaN   16    4    8   20.1  ...  0.2   

   ﾀｲﾑ 指数           通過        ペース    上り      馬体重 厩舎 ｺﾒﾝﾄ  備考  勝ち馬 (2着馬)     賞金  
0      **    13-15-5-6  35.9-36.3  39.5  460(+4)     NaN NaN    タニノヨセミテ    NaN  
1      **    18-18-5-5  35.4-35.9  35.8  456(-4)     NaN NaN       チャイカ  130.0  
2      **  15-14-15-14  37.8-36.7  37.8  460(-4)     NaN NaN  タンジェリンムーン    NaN  
3      **        12-13  35.9-35.1  37.4  464(+2)     NaN NaN   ラルナブリラーレ    NaN  

In [88]:
def create_table_for_predict_():
    """
    scheduled_raceに保管したhtmlをスクレイピング。
    dateはyyyy/mm/ddの形式。
    """

    race_html_list = [
        "202410020110.bin",
        "202410020111.bin",
        "202405010511.bin",
        "202408020510.bin",
        "202410020109.bin",
        "202408020509.bin",
        "202405010510.bin",
        "202405010509.bin",
        "202408020511.bin",
    ]
    # print("race_html_list", race_html_list)

    df = pd.DataFrame()
    for race_html in tqdm(race_html_list):
        race_html_path = os.path.join("./data/html/scheduled_race/", race_html)
        with open(race_html_path, "rb") as f:
            # 保存してあるbinファイルを読み込む
            race_id = race_html[:12]
            html = f.read()
            df = pd.read_html(html)

            # htmlをsoupオブジェクトに変換
            soup = BeautifulSoup(html, "lxml")
            soup_span = soup.find_all("span")
            # メインのテーブルの取得
            for tr in soup.find_all("tr", class_="HorseList"):
                row = []
                for td in tr.find_all("td"):
                    if "HorseInfo" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"horse/(\d*)", href)[0])
                    elif "Jockey" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"jockey/result/recent/(\w*)", href)[0])
                    elif "Trainer" in td.get("class", []):
                        anchor = td.find("a")
                        if anchor:
                            href = anchor.get("href")
                            row.append(re.findall(r"trainer/result/recent/(\w*)", href)[0])
                    # elif "Txt_R Popular" in td.get("class", []):
                    #    span = td.find("span")
                    #    if span:
                    #        row.append(re.findall(r"(?<=>)\d+(\.\d+)?", span.text)[0])

                    row.append(td.text)
                print("row", row)
                # rowをDataFrameに変換し、dfに連結
                # DataFrameのリストを作成
                # 列のラベル
                columns = list(range(16))
                # DataFrameの作成
                df = pd.DataFrame([row], columns=columns)

                print("df", df)

                # レース結果テーブルと列を揃える
            df = df[[0, 1, 5, 6, 12, 13, 11, 3, 7, 9]]
            df.columns = [
                Cols.WAKUBAN,
                Cols.UMABAN,
                Cols.SEX_AGE,
                Cols.KINRYO,
                Cols.TANSHO_ODDS,
                Cols.POPULARITY,
                Cols.WEIGHT_AND_DIFF,
                "horse_id",
                "jockey_id",
                "trainer_id",
            ]
            df.index = [race_id] * len(df)

            print("df", df)


"""


            # レース情報の取得
            texts = soup.find_element(By.CLASS_NAME, "RaceList_Item02").text
            texts = re.findall(r"\w+", texts)
            # 障害レースフラグを初期化
            hurdle_race_flg = False
            for text in texts:
                if "m" in text:
                    # 20211212：[0]→[-1]に修正
                    df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                if text in Master.WEATHER_LIST:
                    df["weather"] = [text] * len(df)
                if text in Master.GROUND_STATE_LIST:
                    df["ground_state"] = [text] * len(df)
                if "稍" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[1]] * len(df)
                if "不" in text:
                    df["ground_state"] = [Master.GROUND_STATE_LIST[3]] * len(df)
                if "芝" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[0]] * len(df)
                if "ダ" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[1]] * len(df)
                if "障" in text:
                    df["race_type"] = [list(Master.RACE_TYPE_DICT.values())[2]] * len(df)
                    hurdle_race_flg = True
                if "右" in text:
                    df["around"] = [Master.AROUND_LIST[0]] * len(df)
                if "左" in text:
                    df["around"] = [Master.AROUND_LIST[1]] * len(df)
                if "直線" in text:
                    df["around"] = [Master.AROUND_LIST[2]] * len(df)
                if "新馬" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[0]] * len(df)
                if "未勝利" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[1]] * len(df)
                if "１勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[2]] * len(df)
                if "２勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[3]] * len(df)
                if "３勝クラス" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[4]] * len(df)
                if "オープン" in text:
                    df["race_class"] = [Master.RACE_CLASS_LIST[5]] * len(df)

            # グレードレース情報の取得
            if len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType3")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType2")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
            elif len(soup.find_elements(By.CLASS_NAME, "Icon_GradeType1")) > 0:
                df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

            # 障害レースの場合
            if hurdle_race_flg:
                df["around"] = [Master.AROUND_LIST[3]] * len(df)
                df["race_class"] = [Master.RACE_CLASS_LIST[9]] * len(df)

            df["date"] = [date] * len(df)
        # except Exception as e:
        #    print(e)
        # finally:
        #    driver.close()
        #    driver.quit()
"""

create_table_for_predict_()

 22%|██▏       | 2/9 [00:00<00:00,  8.50it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2018104534', '\n\n\nアスクキングコング\n\n\n', 'セ6', '55.0', '01174', '\n岩田望\n', '01084', '栗東梅田', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                      4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2018104534  \n\n\nアスクキングコング\n\n\n  セ6  55.0   

      7        8      9     10  11     12      13         14 15  
0  01174  \n岩田望\n  01084  栗東梅田  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2019104675', '\n\n\nアップストローク\n\n\n', 'セ5', '57.0', '01179', '\n菅原明\n', '01008', '美浦杉浦', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']
df   0  1                        2           3                     4   5     6   \
0        \n\n--◎◯▲△☆&#10003消\n\n  2019104675  \n\n\nアップストローク\n\n\n  セ5  57.0   

      7        8      9     10  11     12      13         14 15  
0  01179  \n菅原明\n  01008  美浦杉浦  \n  ---.-  \n**\n  \n\n\n\n      
row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2020102971', '\n\n\nサト

 22%|██▏       | 2/9 [00:00<00:00,  7.26it/s]

row ['', '', '\n\n--◎◯▲△☆&#10003消\n\n', '2021105764', '\n\n\nアドマイヤベル\n\n\n', '牝3', '55.0', '\n\n○○\n\n', '01059', '美浦加藤征', '\n', '---.-', '\n**\n', '\n\n\n\n ', '']


ValueError: 16 columns passed, passed data had 15 columns

In [105]:
def load_file_pkl_():
    target_file_path = "./data/tmp/race_id_list/temp_race_id_list.txt"
    with open(target_file_path, "r") as f:
        try:
            lines = [line.strip() for line in f]
            # 探している数字を文字列として定義する
            target_number = "201006010106"

            # 文字列として比較する
            index = next((i for i, line in enumerate(lines) if line == target_number), None)

            if index is not None:
                if index < len(lines):
                    loaded_list = lines[:index+1]
                    return loaded_list
                else:
                    print("指定したIDがリストの最後にあります。")
            else:
                print("指定したIDがリスト内に見つかりません。")

        except ValueError:
            print("ファイルからの読み込み中にエラーが発生しました。")
            return None


# 関数を呼び出して実行する
result_list = load_file_pkl_()
print("リスト:", result_list)

リスト: ['201006010101', '201006010102', '201006010103', '201006010104', '201006010105', '201006010106']


In [34]:
from src.preparing.process_selector import select_process


def scrape_horse_id_list(self):
    """
    当日出走するhorse_id一覧を取得
    """
    ref_id_list = self.load_file_pkl()
    waiting_time = 10
    driver = prepare_chrome_driver()
    target_id_list = []
    # driver = prepare_chrome_driver()
    data_index = 1

    print("scraping horse_id_list")

    for ref_id in tqdm(ref_id_list):
        try:
            url = self.from_location + ref_id
            driver.get(url)
            wait = WebDriverWait(driver, waiting_time)
            wait.until(EC.presence_of_all_elements_located)
            html = urlopen(url)
            time.sleep(1)
            soup = BeautifulSoup(html, "lxml")
            # この例ではtarget=horse
            target_td_list = soup.find_all("td", attrs={"class": "txt_l"})
            target_ids = [a["href"] for td in target_td_list if (a := td.find("a")) and "/horse/" in a["href"]]
            target_id = [re.search(r"/horse/(\d+)/", href).group(1) for href in target_ids]
            target_id_list.append(target_id)

        except Exception as e:
            print("Error at {}: {}".format(ref_id, e))
            break
        if data_index % self.batch_size == 0:
            self.target_data = [item for sublist in target_id_list for item in sublist]
            self.save_temp_file("horse_id_list")
            self.obtained_last_key = target_id
            target_id_list = []
        data_index += 1
    # リストの要素を昇順にソート
    flat_list = [item for sublist in target_id_list for item in sublist]
    self.target_data = sorted(set(flat_list))
    self.save_temp_file("horse_id_list")
    self.obtained_last_key = target_id
    self.transfer_temp_file()


horse_id_list = select_process(alias="horse_id_list")

処理対象:horse_id_list
self.from_location:  https://db.netkeiba.com/race/
to_temp_location:  ./data/tmp/horse_id_list/
to_location:  ./data/html/horse_id_list/
self.from_local_location:  ./data/html/race_id_list/
self.from_local_file_name:  race_id_list.pkl
batch_size: 5
reloaded_race_id_list.pkl type:  race_id_list.pkl
scraping horse_id_list


  4%|▎         | 115/3168 [03:20<1:28:52,  1.75s/it]


KeyboardInterrupt: 

In [ ]:
def process_pkl_file(self, process_function):
    """
    binファイルを受け取って、テーブルに変換する関数。
    process_functionを入れ替えながら汎用的に使う共通モジュール
    対象ファイルや処理のバッチサイズなどを読み取り、セットの上、処理する
    """

    target_pkl_files = sorted(self.load_file_pkl(self))
    total_batches = (len(target_pkl_files) + self.batch_size - 1) // self.batch_size  # バッチ数の計算
    total_files = len(target_pkl_files)  # 処理対象の全データ数
    print(f"# of input files: {total_files}")
    processed_files = 0  # 処理済みのファイル数
    print(f"start {self.alias} processing")
    # target_data_name = {}
    driver = prepare_chrome_driver()
    # tqdmインスタンスの作成
    pbar = tqdm(total=total_files, desc="Processing Batches", unit="%", unit_scale=True, leave=True)

    for batch_index in range(total_batches):
        start_index = batch_index * self.batch_size
        end_index = min((batch_index + 1) * self.batch_size, len(target_pkl_files))
        batch_target_pkl_files = target_pkl_files[start_index:end_index]

        for ref_id in batch_target_pkl_files:  #
            try:
                temp_df = process_function(ref_id,driver)  # , target_data_name)

            except Exception as e:
                print("Error at {}: {}".format(ref_id, e))
                break

            processed_files += 1
            pbar.update(1)  # 処理済みのファイル数を1増やす
            temp_df = pd.concat([temp_df[key] for key in temp_df])

            # if self.alias == "race_results_table":
            #    self.target_data = trim_function(temp_df)
            self.target_data = temp_df

            self.save_temp_file(self.alias)
            # target_data_name = {}  # バッチ処理が完了したので辞書をクリア
            temp_df = []
        self.obtained_last_key = ref_id[-1]
        self.transfer_temp_file()
    print(f"# of processed files: {processed_files}")



def get_soup(self,ref_id,driver):
    url = self.from_location + ref_id
    driver.get(url)
    wait = WebDriverWait(driver, waiting_time=10)
    wait.until(EC.presence_of_all_elements_located)
    html = urlopen(url)
    time.sleep(1)
    soup = BeautifulSoup(html, "lxml")
    return soup

def get_raw_horse_id_list(ref_id,driver):
    soup = get_soup(ref_id, driver)
    # この例ではtarget=horse
    target_id_list=[]
    target_td_list = soup.find_all("td", attrs={"class": "txt_l"})
    target_ids = [a["href"] for td in target_td_list if (a := td.find("a")) and "/horse/" in a["href"]]
    target_id = [re.search(r"/horse/(\d+)/", href).group(1) for href in target_ids]
    for id in range(len(target_id)):
        target_id_list.append(target_id[id])
    target_id_list.append(target_id)
    return target_id_list

In [14]:
import pandas as pd
from abc import ABCMeta, abstractmethod


class Abstract:
    def __init__(self, filepath: str):
        self.__raw_data = pd.DataFrame(pd.read_pickle(filepath))
        self.__preprocessed_data = self._preprocess()

    @abstractmethod
    def _preprocess(self):
        pass

    @property
    def raw_data(self):
        new_columns = {col: int(col) if str(col).isdigit() else col for col in self.__raw_data.columns}
        # 列名を変換
        self.__raw_data.rename(columns=new_columns, inplace=True)
        return self.__raw_data.copy()

    @property
    def preprocessed_data(self):
        return self.__preprocessed_data.copy()


inst = Abstract("./data/raw/return_tables.pkl")
df = inst.raw_data
# new_columns = {col: int(col) if col.isdigit() else col for col in df.columns if col.isdigit()}
# 列名を変換
# df.rename(columns=new_columns, inplace=True)
print(df)

# 列名のリストを取得
column_names = df.columns

# 列名が整数かどうかをチェック
for name in column_names:
    if isinstance(name, int):
        print(f"列名 {name} は整数です。")
    else:
        print(f"列名 {name} は整数ではありません。")
# print(f"df['0']:{df['0']}")
print(f"df[0]:{df[0]}")

         0                        1                2         3       race_id
0       単勝                       14              240         1  202303010101
1       複勝                14br4br15    140br180br180   2br4br3  202303010101
2       枠連                    3 - 8              490         2  202303010101
3       馬連                   4 - 14              790         2  202303010101
4      ワイド  4 - 14br14 - 15br4 - 15    360br390br600   3br5br8  202303010101
...    ...                      ...              ...       ...           ...
11650   馬連                    2 - 3              940         2  202310020612
11651  ワイド    2 - 3br3 - 16br2 - 16  370br590br1,370  2br4br17  202310020612
11652   馬単                    3 → 2             1270         2  202310020612
11653  三連複               2 - 3 - 16             3700         8  202310020612
11654  三連単               3 → 2 → 16            11950        20  202310020612

[11655 rows x 5 columns]
列名 0 は整数です。
列名 1 は整数です。
列名 2 は整数です。
列名 3 は整数です。
列名

In [249]:
def create_raw_race_info_(target_bin_file_path):
    # print(f"target_bin_file_path:{target_bin_file_path}")
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        texts = (
            soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
            + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        )
        info = re.findall(r"\w+", texts)
        df = pd.DataFrame()
        # 障害レースフラグを初期化
        hurdle_race_flg = False
        for text in info:
            if text in ["芝", "ダート"]:
                df["race_type"] = [text]
            if "障" in text:
                df["race_type"] = ["障害"]
                hurdle_race_flg = True
            if "m" in text:
                # 20211212：[0]→[-1]に修正
                df["course_len"] = [int(re.findall(r"\d+", text)[-1])]
            if text in Master.GROUND_STATE_LIST:
                df["ground_state"] = [text]
            if text in Master.WEATHER_LIST:
                df["weather"] = [text]
            if "年" in text:
                df["date"] = [text]
            if "右" in text:
                df["around"] = [Master.AROUND_LIST[0]]
            if "左" in text:
                df["around"] = [Master.AROUND_LIST[1]]
            if "直線" in text:
                df["around"] = [Master.AROUND_LIST[2]]
            if "新馬" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[0]]
            if "未勝利" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[1]]
            if ("1勝クラス" in text) or ("500万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[2]]
            if ("2勝クラス" in text) or ("1000万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[3]]
            if ("3勝クラス" in text) or ("1600万下" in text):
                df["race_class"] = [Master.RACE_CLASS_LIST[4]]
            if "オープン" in text:
                df["race_class"] = [Master.RACE_CLASS_LIST[5]]

        # グレードレース情報の取得
        grade_text = soup.find("div", attrs={"class": "data_intro"}).find_all("h1")[0].text
        if "G3" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[6]] * len(df)
        elif "G2" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[7]] * len(df)
        elif "G1" in grade_text:
            df["race_class"] = [Master.RACE_CLASS_LIST[8]] * len(df)

        # 障害レースの場合
        if hurdle_race_flg:
            df["around"] = [Master.AROUND_LIST[3]]
            df["race_class"] = [Master.RACE_CLASS_LIST[9]]

        # インデックスをrace_idにする

        race_id = re.findall(r"race\W(\d+).bin", target_bin_file_path)[0]
        df.index = [race_id] * len(df)
        df[race_id] = df.index


create_raw_race_info_("./data/html/race/202006010101.bin")

FileNotFoundError: [Errno 2] No such file or directory: './data/html/race/202006010101.bin'

In [22]:
import re

# 仮のテキストとDataFrame
texts = [
    [
        "芝右",
        "外2400m",
        "天候",
        "曇",
        "芝",
        "良",
        "発走",
        "15",
        "35",
        "過去の日経新春杯",
        "2020年1月19日",
        "1回京都7日目",
        "4歳以上オープン",
        "国際",
        "特指",
        "ハンデ",
    ],
    [
        "ダ右1200m",
        "天候",
        "曇",
        "ダート",
        "良",
        "発走",
        "16",
        "10",
        "2020年1月19日",
        "1回京都7日目",
        "4歳以上2勝クラス",
        "指",
        "定量",
    ],
]
df_list = []

# テキストからコースの長さを抽出してDataFrameに設定する
for text_list in texts:
    course_len = None
    for text in text_list:
        match = re.search(r"\d{3,4}m", text)  # 正規表現でマッチング
        if match:
            course_len = int(match.group()[:-1])  # m を取り除いて整数に変換
            break  # コースの長さが見つかればループを終了
    df_list.append({"course_len": course_len})

print(df_list)

[{'course_len': 2400}, {'course_len': 1200}]


In [196]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
def dart_checker(text1):
    if text1.split("/")[0].strip()[:2] == "障芝":
        dart = True
    else:
        dart = False
    return dart


def count_ground_state(text1):
    return text1.split("/")[2].count(":")


def create_tmp_race_info_(target_bin_file_path):
    from src.constants._master import Master

    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")

        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        text1 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
        text2 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        # print(f"text1:{text1}")
        # print(f"text2:{text2}")
        race_id = re.findall(r"\d+", target_bin_file_path)[0]
        # テキスト情報を解析してDataFrameに変換
        race_distance = re.search(r"\d+", text1.split("/")[0]).group()
        weather = text1.split("/")[1].split(":")[1].strip()

        if weather in Master.WEATHER_LIST:
            pass
        else:
            print(f"unknown weather definition appeared:{race_id}")

        race_type = text1.split("/")[2].split(":")[0].strip()
        # 発走時刻
        start_time = text1.split("/")[-1].split(":")[1:3]
        start_time = ":".join(start_time).strip().split("\n\n")[0]

        race_date = text2.split(" ")[0]
        race_name = text2.split(" ")[1]
        dart = dart_checker(text1)
        print(f"dart:{dart}")
        print(f"count:{count_ground_state(text1)}")
        test = text1.split("/")[2].split(":")[2]
        print(f"gs2:{test}")

        for around in Master.AROUND_LIST:
            if around in text1.split("/")[0]:
                around_info = around
            else:
                around_info = None

        # 開催日数と開催回数を取得
        race_day_count = re.search(r"\d+", race_name.split("日目")[0]).group()  # 開催日数
        race_round_count = re.search(r"\d+", race_name.split("回")[0]).group()  # 開催回数

        # 開催場所を取得
        place_id = None
        for key, value in Master.PLACE_DICT.items():
            if key in race_name:
                place_id = value
                place_name = key
        if count_ground_state(text1) == 1:
            temp_ground_state0 = text1.split("/")[2].split(":")[1].strip()
            if temp_ground_state0 in Master.GROUND_STATE_LIST:
                ground_state1 = temp_ground_state0
                ground_state2 = temp_ground_state0
            else:
                print(f"unknown GROUND_STATE definition appeared1:{race_id}{ground_state1}{ground_state2}")
        elif dart_checker(text1) and count_ground_state(text1) == 2:
            temp_ground_state1 = text1.split("/")[2].split(":")[1].split()[0].strip()
            if temp_ground_state1 in Master.GROUND_STATE_LIST:
                ground_state1 = temp_ground_state1
            else:
                print(f"unknown GROUND_STATE definition appeared2:{race_id}{ground_state1}")
            temp_ground_state2 = text1.split("/")[2].split(":")[2].strip()
            if temp_ground_state2 in Master.GROUND_STATE_LIST:
                ground_state2 = temp_ground_state2
            else:
                print(f"unknown GROUND_STATE definition appeared3:{race_id}{temp_ground_state2}")
        # 不要な部分を削除
        # レース条件から年齢、性別、レースクラスを削除
        # 馬齢を取得
        race_condition = text2.split(" ")[2]

        # レース条件に基づいてフラグを設定
        race_flags = {}
        if race_condition is not None:
            # 性別を取得
            sex_info = None
            for sex in Master.SEX_LIST:
                if sex in race_condition:
                    sex_info = sex

            # レースクラスを取得
            race_class_info = None
            for race_class in Master.RACE_CLASS_LIST:
                if race_class in race_condition:
                    race_class_info = race_class
            if race_class_info is None:
                print(f"unknown race_class definition appeared:{race_id}")
            # 向きを取得

            if (around_info is None) and (("障害" in race_class_info or race_condition) or dart):
                around_info = "直線"

            for key, value in Master.RACE_CONDITION_DICT.items():
                if key in race_condition:
                    race_flags[value] = 1
                    race_condition = race_condition.replace(key, "").strip()
                else:
                    race_flags[value] = 0

        if "歳以上" in race_condition:
            age = re.search(r"\d+", race_condition.split("歳以上")[0]).group() + "+"

        else:
            age = re.search(r"\d+", race_condition.split("歳")[0]).group()
        if race_condition is not None:
            # ageの処理を修正
            if age is not None and age != "":
                if "+" in age:
                    race_condition = race_condition.replace(age[:-1], "").replace("歳以上", "").strip()
                else:
                    race_condition = race_condition.replace(age, "").replace("歳", "").strip()

            if sex_info is not None and sex_info != "":
                race_condition = race_condition.replace(sex_info, "").strip()
            if race_class_info is not None and race_class_info != "":
                race_condition = race_condition.replace(race_class_info, "").strip()
            if race_condition is not None and race_condition != "":
                race_condition = race_condition.replace("()", "").strip()
                race_condition = race_condition.replace("[]", "").strip()
            if race_condition is not None and race_condition != "":
                race_condition = race_condition.strip()

        # DataFrame作成歳
        df = pd.DataFrame(
            {
                # "レース名": [race_name],
                "レース場id": [place_id],
                "レース場名": [place_name],
                "開催日数": [race_day_count],  # 開催日数を追加
                "開催回数": [race_round_count],  # 開催回数を追加
                "レース開催日": [race_date],
                "発走時刻": [start_time],
                "レース種類": [race_type],
                "向き": [around_info],
                "レース距離": [race_distance],
                "天候": [weather],
                "馬場状態1": [ground_state1],
                "馬場状態2": [ground_state2],
                "馬齢": [age],
                "性別": [sex_info],
                "レースクラス": [race_class_info],
                "レース条件": [race_condition],
                **race_flags,
            }
        )
    return df


df = create_tmp_race_info_("./data/html/race/202305021104.bin")
print(df)

dart:True
count:2
gs2: 良 
  レース場id レース場名 開催日数 開催回数      レース開催日   発走時刻 レース種類  向き レース距離 天候  ... teiryo  \
0     05    東京    2    2  2023年5月27日  11:35     芝  直線  3000  晴  ...      1   

  barei bettei kokusai shimei tokushi  toku  handi  minarai  shogai  
0     0      0       0      0       0     0      0        0       1  

[1 rows x 26 columns]


In [37]:
def create_tmp_race_info_(target_bin_file_path):
    # print(f"target_bin_file_path:{target_bin_file_path}")
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()
        # 馬の基本情報を取得
        #df = pd.read_html(html)[1].set_index(0).T
        # df = pd.read_html(html)
        # print(f"df:{df}")
        # htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")
        print(f"soup:{soup}")
        # 天候、レースの種類、コースの長さ、馬場の状態、日付、回り、レースクラスをスクレイピング
        #text1 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
        #text2 = soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
        ##print(f"text1:{text1}")
        #print(f"text2:{text2}")
        # info = re.findall(r"\w+", texts)
        # race_id = re.findall(r"\d+", target_bin_file_path)[0]
        # print(race_id, info)

    return


create_tmp_race_info_("./data/html/race/202303010101.bin")

soup:<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>3歳未勝利｜2023年4月8日 | 競馬データベース - netkeiba.com</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeiba.comが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba.com" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeiba.comが誇る国内最大級の競馬デ

In [46]:
from collections import Counter


def get_ids(df):
    # 空のDataFrameを作成
    ids_df = pd.DataFrame(columns=["race_id", "weather_id", "race_time_id", "race_date_id", "diff"])
    # 各race_idごとに処理を行う
    for race_id, group in df.groupby("race_id"):
        # "天候"が含まれている最初の行のidを記録する
        weather_id = group[group["info"].str.contains("天候")]["id"].iloc[0]
        # "発走"が含まれている最初の行のidを記録する
        race_time_id = group[group["info"].str.contains("発走")]["id"].iloc[0]
        race_date_id = group[group["info"].str.contains(r"\d{1,4}年\d{1,2}月\d{1,2}日")]["id"].iloc[0]
        # DataFrameに情報を追加
        ids_df = pd.concat(
            [
                ids_df,
                pd.DataFrame(
                    {
                        "race_id": [race_id],
                        "weather_id": [weather_id],
                        "race_time_id": [race_time_id],
                        "race_date_id": [race_date_id],
                        "diff": [race_time_id][0] - [weather_id][0],
                    }
                ),
            ]
        )

    return ids_df


def count_unique_values(df, column_name):
    """
    DataFrameの指定された列に含まれるユニークな値とその出現回数をカウントする関数。
    """
    # 指定された列のユニークな値を取得し、出現回数をカウントする
    unique_values_counts = df[column_name].value_counts()
    return unique_values_counts


df = pd.read_csv("./data/tmp/tmp_for_race_info/tmp_race_info_table.csv")
df = df.sort_values(by=["race_id", "id"])

print(f"df: {df.head(20)}")
columns = get_ids(df)
print(columns.tail())
print("max",df["id"].max())

# ユニークな値とその出現回数をカウントする列のリスト
columns_to_count = [
    "weather_id",
    "race_time_id",
    "race_date_id",
    "diff",

]  # 任意の列を追加可能

# 各列に対してユニークな値とその出現回数を取得し、表示する
for column_name in columns_to_count:
    print(f"{column_name}の内訳:")
    counts = count_unique_values(columns, column_name)
    print(counts)
    print()

# 4つの変数をzipして組み合わせを作成します
combined_variables = zip(
    columns["weather_id"], columns["race_time_id"], columns["race_date_id"]
)

# 組み合わせの数をカウントします
combination_counts = Counter(combined_variables)

# 結果を出力します
for combination, count in combination_counts.items():
    print(combination, ":", count)

df:     id       info       race_id
0    1    ダ右1700m  202303010101
1    2         天候  202303010101
2    3          曇  202303010101
3    4        ダート  202303010101
4    5          良  202303010101
5    6         発走  202303010101
6    7         09  202303010101
7    8         50  202303010101
8    9  2023年4月8日  202303010101
9   10    1回福島1日目  202303010101
10  11      3歳未勝利  202303010101
11  12          牝  202303010101
12  13          指  202303010101
13  14         馬齢  202303010101
14   1    芝右1200m  202303010102
15   2         天候  202303010102
16   3          曇  202303010102
17   4          芝  202303010102
18   5          良  202303010102
19   6         発走  202303010102
        race_id weather_id race_time_id race_date_id diff
0  202310020608          2            6            9    4
0  202310020609          2            6           10    4
0  202310020610          2            6           10    4
0  202310020611          2            6           10    4
0  202310020612          2        

In [67]:
df_info = pd.read_csv("./data/tmp/tmp_for_race_info/tmp_race_info_table.csv")
df_info = df_info.sort_values(by=["race_id", "id"])
df = pd.DataFrame()
print(f"df: {df_info.head(20)}")


# race_idをキーにして、"天候"を含む情報をリストにまとめる
def extract_weather_info(group):
    if any(group.iloc[2:5, :].eq("天候").any()):
        return [group.iloc[:, 1].values.tolist()]
    else:
        return None


# "天候"というinfoを持つ行のidを取得
weather_ids = df_info.loc[df_info["info"] == "天候", "id"]

# 対応するrace_idを取得
race_ids = df_info.loc[df_info["id"].isin(weather_ids), "race_id"]

# race_idをキーにして、対応するidを並べる
df = pd.DataFrame({"race_id": race_ids, "id": weather_ids}).groupby("race_id")["id"].apply(list).reset_index(name="ids")

print(df)


df:     id       info       race_id
0    1    ダ右1700m  202303010101
1    2         天候  202303010101
2    3          曇  202303010101
3    4        ダート  202303010101
4    5          良  202303010101
5    6         発走  202303010101
6    7         09  202303010101
7    8         50  202303010101
8    9  2023年4月8日  202303010101
9   10    1回福島1日目  202303010101
10  11      3歳未勝利  202303010101
11  12          牝  202303010101
12  13          指  202303010101
13  14         馬齢  202303010101
14   1    芝右1200m  202303010102
15   2         天候  202303010102
16   3          曇  202303010102
17   4          芝  202303010102
18   5          良  202303010102
19   6         発走  202303010102
           race_id              ids
0     202303010101  [2.0, nan, nan]
1     202303010102  [2.0, nan, nan]
2     202303010103  [2.0, nan, nan]
3     202303010104  [2.0, nan, nan]
4     202303010105  [2.0, nan, nan]
...            ...              ...
1459  202310020608  [2.0, nan, nan]
1460  202310020609  [2.0, nan, nan]


In [30]:
import pickle


def read_pkl_file(file_path):
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    return data


# pklファイルのパスを指定してデータを読み込む
file_path = "./data/html/race_id_list/race_id_list.pkl"
pkl_data = read_pkl_file(file_path)
if pkl_data is not None:
    # 対象のデータがDataFrame内に存在するか確認し、位置を取得
    target_number = "202303010102"
    print(f"pkl_data:{pkl_data}")
    for index, value in pkl_data["race_id"].items():
        value_stripped = str(value).strip()
        if value_stripped == target_number:
            print(f"ここにあるよ: {index}")
            break
    else:
        print("対象のデータが見つかりませんでした。")
else:
    print("DataFrameがNoneを返しました。ファイルの読み込みに問題がある可能性があります。")

pkl_data:           race_id
0     202303010101
1     202303010102
2     202303010103
3     202303010104
4     202303010105
...            ...
1459  202310020608
1460  202310020609
1461  202310020610
1462  202310020611
1463  202310020612

[1464 rows x 1 columns]
ここにあるよ: 1


In [5]:
import pandas as pd
target_all_files = pd.date_range(start="2023-01-01", end="2023-05-02", freq="ME")
len(target_all_files)

4

In [6]:
import pandas as pd

# 例としてDataFrameを作成します
data = {"A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
df = pd.DataFrame(data)

# 列番号で昇順にソートします
sorted_columns = sorted(df.columns)
sorted_df = df[sorted_columns]

print(sorted_df)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9


In [31]:
import pandas as pd

# サンプルのDataFrameを作成する
data = {"A": [1, 2, 3], "B": [4, 5, 6], "C": [7, 8, 9]}
df = pd.DataFrame(data)

# 最後の列を選択する
last_column = df.iloc[:, -1]
print(last_column)

0    7
1    8
2    9
Name: C, dtype: int64


In [53]:
import pandas as pd

# サンプルのDataFrame作成
data = {
    "race_id": [1, 1, 1, 2, 2, 2],
    "id": [1, 2, 3, 1, 2, 3],
    "col1": ["info1", "info2","info4", "info5", "info6"],
    "col2": ["天候", "info8", "info9","info11", "info12"],
    "col3": ["info13", "info15", "info16", "info17", "天候"],
}

df_info = pd.DataFrame(data)

# race_idをキーにして、"天候"を含む情報をリストにまとめる
df = (
    df_info.groupby("race_id")
    .apply(lambda x: x.iloc[:, 2:].values.tolist() if any(x.iloc[:, 2:5].eq("天候").any()) else None)
    .reset_index(name="天候")
)

print(df)

ValueError: All arrays must be of the same length

In [9]:
# import os
import pandas as pd
from numpy import NaN

# from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re
# from modules.constants import Master


def create_raw_horse_info_(target_bin_file_path):
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # 馬の基本情報を取得
        df = pd.read_html(html)[1].set_index(0).T
        # 馬主IDをスクレイピング

        # 列に "募集情報" が含まれているかを調べる
        funding_info = df.apply(lambda x: x.str.contains("募集情報")).any()
        # 列に "募集情報" がある場合、その列の値を "募集情報" 列に代入する
        if funding_info.any():
            df["募集情報"] = df.loc[:, funding_info].values.flatten()
        else:
            # 列に "募集情報" が含まれていない場合、"募集情報" 列に NaN を入れる
            df["募集情報"] = NaN


"""
# htmlをsoupオブジェクトに変換
        soup = BeautifulSoup(html, "lxml")
        print(f"soup:{soup}")
        user_input = input()
        if user_input == " ":
            pass

        # 調教師IDをスクレイピング
        try:
            trainer_a_list = soup.find("table", attrs={"summary": "のプロフィール"}).find_all(
                "a", attrs={"href": re.compile("^/trainer")}
            )
            trainer_id = re.findall(r"trainer/(\w*)", trainer_a_list[0]["href"])[0]
        except IndexError:
            # 調教師IDを取得できない場合
            # print('trainer_id empty {}'.format(race_html))
            trainer_id = NaN
        df["trainer_id"] = trainer_id

        # 馬主IDをスクレイピング
        try:
            owner_a_list = soup.find("table", attrs={"summary": "のプロフィール"}).find_all(
                "a", attrs={"href": re.compile("^/owner")}
            )
            owner_id = re.findall(r"owner/(\w*)", owner_a_list[0]["href"])[0]
        except IndexError:
            # 馬主IDを取得できない場合
            # print('owner_id empty {}'.format(race_html))
            owner_id = NaN
        df["owner_id"] = owner_id

        # 生産者IDをスクレイピング
        try:
            breeder_a_list = soup.find("table", attrs={"summary": "のプロフィール"}).find_all(
                "a", attrs={"href": re.compile("^/breeder")}
            )
            breeder_id = re.findall(r"breeder/(\w*)", breeder_a_list[0]["href"])[0]
        except IndexError:
            # 生産者IDを取得できない場合
            # print('breeder_id empty {}'.format(race_html))
            breeder_id = NaN
        df["breeder_id"] = breeder_id

        # インデックスをhorse_idにする
        horse_id = re.findall(r"\d+", target_bin_file_path)[0]
        df.index = [horse_id] * len(df)
        df[horse_id] = df.index
"""


create_raw_horse_info_("./data/html/horse/2012103532.bin")

In [13]:
# import os
import pandas as pd
from numpy import NaN

# from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re


def create_raw_horse_info_(target_bin_file_path):
    with open(target_bin_file_path, "rb") as f:
        # 保存してあるbinファイルを読み込む
        html = f.read()

        # 馬の基本情報を取得
        df = pd.read_html(html)[1].set_index(0).T
        # 馬主IDをスクレイピング


        # 列に "募集情報" が含まれているかを調べる
        funding_info = df.apply(lambda x: x.str.contains("募集情報")).any()

        # 列に "募集情報" がある場合、その列の値を "募集情報" 列に代入する
        if funding_info.any():
            df["募集情報"] = df.loc[:, funding_info].values.flatten()
        else:
            # 列に "募集情報" が含まれていない場合、"募集情報" 列に NaN を入れる
            df["募集情報"] = NaN


create_raw_horse_info_("./data/html/horse/2011106610.bin")